In [1]:
import os
import sys

import pandas as pd
import numpy as np
from datetime import datetime as dt
import calendar
import ee
import matplotlib.pyplot as plt

ee.Initialize()

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
def get_dates(img):
    timestamp = ee.Date(img.get('system:time_end'))
    #datetime = dt.fromtimestamp(img.get("system:time_start").getInfo() / 1000.)
    return img.set('timestamp', timestamp)\
              .set('year', timestamp.format('YYYY'))\
              .set('month', timestamp.format('MM'))\
              .set('doy', timestamp.format('D'))

def get_transform(img):
    projection = img.projection()
    json = ee.Dictionary(ee.Algorithms.Describe(projection))
    return ee.List(json.get('transform'))

In [5]:
def get_qa_bits(image, start, end, newName):
    #Function that returns an image containing just the specified QA bits.

    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start, end + 1):
        pattern = pattern + 2**i

    # Return a single band image of the extracted QA bits, 
    #giving the band a new name.
    return image.select([0],[newName]).bitwiseAnd(pattern).rightShift(start)

def qa_mask_modis_FparLai(image):
    algorithm = get_qa_bits(image.select(['FparLai_QC']), 5, 7, 'algorithm').lt(2)
    cloud = get_qa_bits(image.select(['FparLai_QC']), 3, 4, 'cloud').eq(0)
    out = image.updateMask(algorithm).updateMask(cloud)
    return ee.Image(out.copyProperties(image))

def get_buffer(ft, r=1500):
    lon = ft.geometry().coordinates().get(0)
    lat = ft.geometry().coordinates().get(1)
    return ft.buffer(r).copyProperties(ft).set('Lat',lat).set('Lon',lon)

In [6]:
def extract_data():
    data_collection = ee.ImageCollection(product).filterDate(start, end).map(get_dates)
    ex_data = data_collection.map(get_im_data)
    ex_data = ex_data.flatten()
    return ex_data

def extract_data_modis_buffer():
    data_collection = ee.ImageCollection(product)\
                        .filterDate(start, end)\
                        .map(get_dates)\
                        .map(qa_mask_modis_FparLai)
    
    ex_data = data_collection.map(get_im_data)
    ex_data = ex_data.flatten()
    return ex_data

In [7]:
def get_im_data(img):
    #crs = img.projection().crs()
    #crs_transform = get_transform(img)
    #data = img.select(bands).reduceRegions(pts, ee.Reducer.first(), None, crs, crs_transform)
    #data = img.select(bands).reduceRegions(pts, ee.Reducer.first(), None, None, None)
    
    data = img.select(bands).reduceRegions(pts, ee.Reducer.mean(), None, None, None)
    data = data.map(lambda ft: ft.set('year', img.get('year'))\
                                 .set('timestamp', img.get('timestamp'))\
                                 .set('month', img.get('month'))\
                                 .set('doy', img.get('doy')))
    return data

In [8]:
def conv_pt_data(ft, keys):
    try:
        [lon, lat] = ft['geometry']['coordinates']
        ri = [lat, lon]
    except:
        ri = [ft['properties']['Lat'], ft['properties']['Lon']]
    
    for k in keys[2:]:
        if k in ft['properties']:
            ri.append(ft['properties'][k])
        else:
            ri.append(np.nan)

    return ri

In [9]:
def convert_to_df(ee_data, keys):
    data = [conv_pt_data(ft, keys) for ft in ee_data.getInfo()['features']]
    if 'Lon' not in keys:
        keys.append('Lon')
    if 'Lat' not in keys:
        keys.append('Lat')
    df = pd.DataFrame(data=data, columns=keys)
    df['timestamp'] = [dt.fromtimestamp(ts['value'] / 1000.) for ts in df['timestamp']]
    return df            

In [10]:
def extract_pt_by_pt(locations, product_name, bands):
    im = ee.Image(product_name)
    projection = im.projection().getInfo()['crs']
    
    store = []
    columns = ['Lon', 'Lat']
    for band_name in bands:
        columns.append(band_name)
        
    for lon, lat in locations:
        #print(lon, lat)
        point = ee.Geometry.Point(lon, lat)
        bv = [lon, lat]
        for band_name in bands:
            data = im.select(band_name)\
                .reduceRegion(ee.Reducer.first(), point, 1, crs=projection)\
                .get(band_name)
            bv.append(data.getInfo())
        store.append(bv)

    df = pd.DataFrame(data=store, columns=columns)
    return df

In [11]:
data_dir = '/Users/maoyabassiouni/Documents/DATA/Projects/OLNI'

# Get locations (lat/lon)

In [12]:
df_locs = pd.read_csv(os.path.join(data_dir, 'sites', 'all_icp_sites.csv'))
locations = list(zip(df_locs['lon'], df_locs['lat']))

In [13]:
df_locs.head(5)

,lat,lon,code_tree_species,year,n,ymin,ymax,count,yrange
0,34.929444,32.931111,121.0,2008.0,9.77250,2005,2011,4,6
1,34.929444,32.931111,129.0,2008.0,8.69625,2005,2011,4,6
2,34.952222,32.830556,129.0,2008.0,7.95125,2005,2011,4,6
3,35.013889,33.050000,121.0,2008.0,11.11625,2005,2011,4,6
4,35.020000,32.627500,121.0,2008.0,9.71375,2005,2011,4,6


In [14]:
len(locations)

996

In [15]:
period_y = [1990, 2016]

In [16]:
b = 9

# Extract ERA5 meteo

In [34]:
product = 'ECMWF/ERA5/DAILY'
bands = ['total_precipitation', 'surface_pressure', 'mean_2m_air_temperature', ]

In [35]:
for yy in range( period_y[0], period_y[1] + 1):
#yy = 1990
    start = '%s-01-01' % yy
    end = '%s-01-01' % (yy + 1)
    c = []
    for ix in range(np.int(len(locations)/b+1)):
        print(yy, 'batch', ix, ix*b, ix*b+b)
        pts = ee.FeatureCollection([ee.Feature(ee.Geometry.Point([lon, lat])) \
                    for ix, (lon, lat) in enumerate(locations[ix*b: ix*b+b])])
        ee_data = extract_data()
        keys = ['Lat', 'Lon', 'timestamp', 'year', 'month', 'doy']
        for band in bands:
            keys.append(band)
        c.append(convert_to_df(ee_data, keys))
    df = pd.concat(c)
    df = df.reset_index()
    df = df.drop(['index', ], axis=1)
    df.to_csv(os.path.join(data_dir, 'grid_data_extract/ERA5_extract', 'era5_sites_%s.csv' % (yy)))

1990 batch 0 0 9
1990 batch 1 9 18
1990 batch 2 18 27
1990 batch 3 27 36
1990 batch 4 36 45
1990 batch 5 45 54
1990 batch 6 54 63
1990 batch 7 63 72
1990 batch 8 72 81
1990 batch 9 81 90
1990 batch 10 90 99
1990 batch 11 99 108
1990 batch 12 108 117
1990 batch 13 117 126
1990 batch 14 126 135
1990 batch 15 135 144
1990 batch 16 144 153
1990 batch 17 153 162
1990 batch 18 162 171
1990 batch 19 171 180
1990 batch 20 180 189
1990 batch 21 189 198
1990 batch 22 198 207
1990 batch 23 207 216
1990 batch 24 216 225
1990 batch 25 225 234
1990 batch 26 234 243
1990 batch 27 243 252
1990 batch 28 252 261
1990 batch 29 261 270
1990 batch 30 270 279
1990 batch 31 279 288
1990 batch 32 288 297
1990 batch 33 297 306
1990 batch 34 306 315
1990 batch 35 315 324
1990 batch 36 324 333
1990 batch 37 333 342
1990 batch 38 342 351
1990 batch 39 351 360
1990 batch 40 360 369
1990 batch 41 369 378
1990 batch 42 378 387
1990 batch 43 387 396
1990 batch 44 396 405
1990 batch 45 405 414
1990 batch 46 414 423
19

1993 batch 45 405 414
1993 batch 46 414 423
1993 batch 47 423 432
1993 batch 48 432 441
1993 batch 49 441 450
1993 batch 50 450 459
1993 batch 51 459 468
1993 batch 52 468 477
1993 batch 53 477 486
1993 batch 54 486 495
1993 batch 55 495 504
1993 batch 56 504 513
1993 batch 57 513 522
1993 batch 58 522 531
1993 batch 59 531 540
1993 batch 60 540 549
1993 batch 61 549 558
1993 batch 62 558 567
1993 batch 63 567 576
1993 batch 64 576 585
1993 batch 65 585 594
1993 batch 66 594 603
1993 batch 67 603 612
1993 batch 68 612 621
1993 batch 69 621 630
1993 batch 70 630 639
1993 batch 71 639 648
1993 batch 72 648 657
1993 batch 73 657 666
1993 batch 74 666 675
1993 batch 75 675 684
1993 batch 76 684 693
1993 batch 77 693 702
1993 batch 78 702 711
1993 batch 79 711 720
1993 batch 80 720 729
1993 batch 81 729 738
1993 batch 82 738 747
1993 batch 83 747 756
1993 batch 84 756 765
1993 batch 85 765 774
1993 batch 86 774 783
1993 batch 87 783 792
1993 batch 88 792 801
1993 batch 89 801 810
1993 batch

1996 batch 88 792 801
1996 batch 89 801 810
1996 batch 90 810 819
1996 batch 91 819 828
1996 batch 92 828 837
1996 batch 93 837 846
1996 batch 94 846 855
1996 batch 95 855 864
1996 batch 96 864 873
1996 batch 97 873 882
1996 batch 98 882 891
1996 batch 99 891 900
1996 batch 100 900 909
1996 batch 101 909 918
1996 batch 102 918 927
1996 batch 103 927 936
1996 batch 104 936 945
1996 batch 105 945 954
1996 batch 106 954 963
1996 batch 107 963 972
1996 batch 108 972 981
1996 batch 109 981 990
1996 batch 110 990 999
1997 batch 0 0 9
1997 batch 1 9 18
1997 batch 2 18 27
1997 batch 3 27 36
1997 batch 4 36 45
1997 batch 5 45 54
1997 batch 6 54 63
1997 batch 7 63 72
1997 batch 8 72 81
1997 batch 9 81 90
1997 batch 10 90 99
1997 batch 11 99 108
1997 batch 12 108 117
1997 batch 13 117 126
1997 batch 14 126 135
1997 batch 15 135 144
1997 batch 16 144 153
1997 batch 17 153 162
1997 batch 18 162 171
1997 batch 19 171 180
1997 batch 20 180 189
1997 batch 21 189 198
1997 batch 22 198 207
1997 batch 23

2000 batch 21 189 198
2000 batch 22 198 207
2000 batch 23 207 216
2000 batch 24 216 225
2000 batch 25 225 234
2000 batch 26 234 243
2000 batch 27 243 252
2000 batch 28 252 261
2000 batch 29 261 270
2000 batch 30 270 279
2000 batch 31 279 288
2000 batch 32 288 297
2000 batch 33 297 306
2000 batch 34 306 315
2000 batch 35 315 324
2000 batch 36 324 333
2000 batch 37 333 342
2000 batch 38 342 351
2000 batch 39 351 360
2000 batch 40 360 369
2000 batch 41 369 378
2000 batch 42 378 387
2000 batch 43 387 396
2000 batch 44 396 405
2000 batch 45 405 414
2000 batch 46 414 423
2000 batch 47 423 432
2000 batch 48 432 441
2000 batch 49 441 450
2000 batch 50 450 459
2000 batch 51 459 468
2000 batch 52 468 477
2000 batch 53 477 486
2000 batch 54 486 495
2000 batch 55 495 504
2000 batch 56 504 513
2000 batch 57 513 522
2000 batch 58 522 531
2000 batch 59 531 540
2000 batch 60 540 549
2000 batch 61 549 558
2000 batch 62 558 567
2000 batch 63 567 576
2000 batch 64 576 585
2000 batch 65 585 594
2000 batch

2003 batch 64 576 585
2003 batch 65 585 594
2003 batch 66 594 603
2003 batch 67 603 612
2003 batch 68 612 621
2003 batch 69 621 630
2003 batch 70 630 639
2003 batch 71 639 648
2003 batch 72 648 657
2003 batch 73 657 666
2003 batch 74 666 675
2003 batch 75 675 684
2003 batch 76 684 693
2003 batch 77 693 702
2003 batch 78 702 711
2003 batch 79 711 720
2003 batch 80 720 729
2003 batch 81 729 738
2003 batch 82 738 747
2003 batch 83 747 756
2003 batch 84 756 765
2003 batch 85 765 774
2003 batch 86 774 783
2003 batch 87 783 792
2003 batch 88 792 801
2003 batch 89 801 810
2003 batch 90 810 819
2003 batch 91 819 828
2003 batch 92 828 837
2003 batch 93 837 846
2003 batch 94 846 855
2003 batch 95 855 864
2003 batch 96 864 873
2003 batch 97 873 882
2003 batch 98 882 891
2003 batch 99 891 900
2003 batch 100 900 909
2003 batch 101 909 918
2003 batch 102 918 927
2003 batch 103 927 936
2003 batch 104 936 945
2003 batch 105 945 954
2003 batch 106 954 963
2003 batch 107 963 972
2003 batch 108 972 981
2

2006 batch 107 963 972
2006 batch 108 972 981
2006 batch 109 981 990
2006 batch 110 990 999
2007 batch 0 0 9
2007 batch 1 9 18
2007 batch 2 18 27
2007 batch 3 27 36
2007 batch 4 36 45
2007 batch 5 45 54
2007 batch 6 54 63
2007 batch 7 63 72
2007 batch 8 72 81
2007 batch 9 81 90
2007 batch 10 90 99
2007 batch 11 99 108
2007 batch 12 108 117
2007 batch 13 117 126
2007 batch 14 126 135
2007 batch 15 135 144
2007 batch 16 144 153
2007 batch 17 153 162
2007 batch 18 162 171
2007 batch 19 171 180
2007 batch 20 180 189
2007 batch 21 189 198
2007 batch 22 198 207
2007 batch 23 207 216
2007 batch 24 216 225
2007 batch 25 225 234
2007 batch 26 234 243
2007 batch 27 243 252
2007 batch 28 252 261
2007 batch 29 261 270
2007 batch 30 270 279
2007 batch 31 279 288
2007 batch 32 288 297
2007 batch 33 297 306
2007 batch 34 306 315
2007 batch 35 315 324
2007 batch 36 324 333
2007 batch 37 333 342
2007 batch 38 342 351
2007 batch 39 351 360
2007 batch 40 360 369
2007 batch 41 369 378
2007 batch 42 378 38

2010 batch 41 369 378
2010 batch 42 378 387
2010 batch 43 387 396
2010 batch 44 396 405
2010 batch 45 405 414
2010 batch 46 414 423
2010 batch 47 423 432
2010 batch 48 432 441
2010 batch 49 441 450
2010 batch 50 450 459
2010 batch 51 459 468
2010 batch 52 468 477
2010 batch 53 477 486
2010 batch 54 486 495
2010 batch 55 495 504
2010 batch 56 504 513
2010 batch 57 513 522
2010 batch 58 522 531
2010 batch 59 531 540
2010 batch 60 540 549
2010 batch 61 549 558
2010 batch 62 558 567
2010 batch 63 567 576
2010 batch 64 576 585
2010 batch 65 585 594
2010 batch 66 594 603
2010 batch 67 603 612
2010 batch 68 612 621
2010 batch 69 621 630
2010 batch 70 630 639
2010 batch 71 639 648
2010 batch 72 648 657
2010 batch 73 657 666
2010 batch 74 666 675
2010 batch 75 675 684
2010 batch 76 684 693
2010 batch 77 693 702
2010 batch 78 702 711
2010 batch 79 711 720
2010 batch 80 720 729
2010 batch 81 729 738
2010 batch 82 738 747
2010 batch 83 747 756
2010 batch 84 756 765
2010 batch 85 765 774
2010 batch

2013 batch 84 756 765
2013 batch 85 765 774
2013 batch 86 774 783
2013 batch 87 783 792
2013 batch 88 792 801
2013 batch 89 801 810
2013 batch 90 810 819
2013 batch 91 819 828
2013 batch 92 828 837
2013 batch 93 837 846
2013 batch 94 846 855
2013 batch 95 855 864
2013 batch 96 864 873
2013 batch 97 873 882
2013 batch 98 882 891
2013 batch 99 891 900
2013 batch 100 900 909
2013 batch 101 909 918
2013 batch 102 918 927
2013 batch 103 927 936
2013 batch 104 936 945
2013 batch 105 945 954
2013 batch 106 954 963
2013 batch 107 963 972
2013 batch 108 972 981
2013 batch 109 981 990
2013 batch 110 990 999
2014 batch 0 0 9
2014 batch 1 9 18
2014 batch 2 18 27
2014 batch 3 27 36
2014 batch 4 36 45
2014 batch 5 45 54
2014 batch 6 54 63
2014 batch 7 63 72
2014 batch 8 72 81
2014 batch 9 81 90
2014 batch 10 90 99
2014 batch 11 99 108
2014 batch 12 108 117
2014 batch 13 117 126
2014 batch 14 126 135
2014 batch 15 135 144
2014 batch 16 144 153
2014 batch 17 153 162
2014 batch 18 162 171
2014 batch 19

# Extract terraclimate meteo

In [36]:
product = 'IDAHO_EPSCOR/TERRACLIMATE'
bands = ['srad', 'pet', 'aet', 'def', 'pdsi', 'pr',  'ro', 'soil', 'tmmn', 'tmmx', 'vap', 'vpd', 'vs']

In [37]:
for yy in range(period_y[0], period_y[1] + 1):
    start = '%s-01-01' % yy
    end = '%s-01-01' % (yy + 1)
    c = []
    for ix in range(np.int(len(locations)/b+1)):
        print(yy, 'batch', ix, ix*b, ix*b+b)
        pts = ee.FeatureCollection([ee.Feature(ee.Geometry.Point([lon, lat])) \
                    for ix, (lon, lat) in enumerate(locations[ix*b: ix*b+b])])
        ee_data = extract_data()
        keys = ['Lat', 'Lon', 'timestamp', 'year', 'month', 'doy']
        for band in bands:
            keys.append(band)
        c.append(convert_to_df(ee_data, keys))
    df = pd.concat(c)
    df = df.reset_index()
    df = df.drop(['index', ], axis=1)
    df.to_csv(os.path.join(data_dir, 'grid_data_extract/TC_extract', 'TC_sites_%s.csv' % (yy)))

1990 batch 0 0 9
1990 batch 1 9 18
1990 batch 2 18 27
1990 batch 3 27 36
1990 batch 4 36 45
1990 batch 5 45 54
1990 batch 6 54 63
1990 batch 7 63 72
1990 batch 8 72 81
1990 batch 9 81 90
1990 batch 10 90 99
1990 batch 11 99 108
1990 batch 12 108 117
1990 batch 13 117 126
1990 batch 14 126 135
1990 batch 15 135 144
1990 batch 16 144 153
1990 batch 17 153 162
1990 batch 18 162 171
1990 batch 19 171 180
1990 batch 20 180 189
1990 batch 21 189 198
1990 batch 22 198 207
1990 batch 23 207 216
1990 batch 24 216 225
1990 batch 25 225 234
1990 batch 26 234 243
1990 batch 27 243 252
1990 batch 28 252 261
1990 batch 29 261 270
1990 batch 30 270 279
1990 batch 31 279 288
1990 batch 32 288 297
1990 batch 33 297 306
1990 batch 34 306 315
1990 batch 35 315 324
1990 batch 36 324 333
1990 batch 37 333 342
1990 batch 38 342 351
1990 batch 39 351 360
1990 batch 40 360 369
1990 batch 41 369 378
1990 batch 42 378 387
1990 batch 43 387 396
1990 batch 44 396 405
1990 batch 45 405 414
1990 batch 46 414 423
19

1993 batch 46 414 423
1993 batch 47 423 432
1993 batch 48 432 441
1993 batch 49 441 450
1993 batch 50 450 459
1993 batch 51 459 468
1993 batch 52 468 477
1993 batch 53 477 486
1993 batch 54 486 495
1993 batch 55 495 504
1993 batch 56 504 513
1993 batch 57 513 522
1993 batch 58 522 531
1993 batch 59 531 540
1993 batch 60 540 549
1993 batch 61 549 558
1993 batch 62 558 567
1993 batch 63 567 576
1993 batch 64 576 585
1993 batch 65 585 594
1993 batch 66 594 603
1993 batch 67 603 612
1993 batch 68 612 621
1993 batch 69 621 630
1993 batch 70 630 639
1993 batch 71 639 648
1993 batch 72 648 657
1993 batch 73 657 666
1993 batch 74 666 675
1993 batch 75 675 684
1993 batch 76 684 693
1993 batch 77 693 702
1993 batch 78 702 711
1993 batch 79 711 720
1993 batch 80 720 729
1993 batch 81 729 738
1993 batch 82 738 747
1993 batch 83 747 756
1993 batch 84 756 765
1993 batch 85 765 774
1993 batch 86 774 783
1993 batch 87 783 792
1993 batch 88 792 801
1993 batch 89 801 810
1993 batch 90 810 819
1993 batch

1996 batch 89 801 810
1996 batch 90 810 819
1996 batch 91 819 828
1996 batch 92 828 837
1996 batch 93 837 846
1996 batch 94 846 855
1996 batch 95 855 864
1996 batch 96 864 873
1996 batch 97 873 882
1996 batch 98 882 891
1996 batch 99 891 900
1996 batch 100 900 909
1996 batch 101 909 918
1996 batch 102 918 927
1996 batch 103 927 936
1996 batch 104 936 945
1996 batch 105 945 954
1996 batch 106 954 963
1996 batch 107 963 972
1996 batch 108 972 981
1996 batch 109 981 990
1996 batch 110 990 999
1997 batch 0 0 9
1997 batch 1 9 18
1997 batch 2 18 27
1997 batch 3 27 36
1997 batch 4 36 45
1997 batch 5 45 54
1997 batch 6 54 63
1997 batch 7 63 72
1997 batch 8 72 81
1997 batch 9 81 90
1997 batch 10 90 99
1997 batch 11 99 108
1997 batch 12 108 117
1997 batch 13 117 126
1997 batch 14 126 135
1997 batch 15 135 144
1997 batch 16 144 153
1997 batch 17 153 162
1997 batch 18 162 171
1997 batch 19 171 180
1997 batch 20 180 189
1997 batch 21 189 198
1997 batch 22 198 207
1997 batch 23 207 216
1997 batch 24

2000 batch 22 198 207
2000 batch 23 207 216
2000 batch 24 216 225
2000 batch 25 225 234
2000 batch 26 234 243
2000 batch 27 243 252
2000 batch 28 252 261
2000 batch 29 261 270
2000 batch 30 270 279
2000 batch 31 279 288
2000 batch 32 288 297
2000 batch 33 297 306
2000 batch 34 306 315
2000 batch 35 315 324
2000 batch 36 324 333
2000 batch 37 333 342
2000 batch 38 342 351
2000 batch 39 351 360
2000 batch 40 360 369
2000 batch 41 369 378
2000 batch 42 378 387
2000 batch 43 387 396
2000 batch 44 396 405
2000 batch 45 405 414
2000 batch 46 414 423
2000 batch 47 423 432
2000 batch 48 432 441
2000 batch 49 441 450
2000 batch 50 450 459
2000 batch 51 459 468
2000 batch 52 468 477
2000 batch 53 477 486
2000 batch 54 486 495
2000 batch 55 495 504
2000 batch 56 504 513
2000 batch 57 513 522
2000 batch 58 522 531
2000 batch 59 531 540
2000 batch 60 540 549
2000 batch 61 549 558
2000 batch 62 558 567
2000 batch 63 567 576
2000 batch 64 576 585
2000 batch 65 585 594
2000 batch 66 594 603
2000 batch

2003 batch 65 585 594
2003 batch 66 594 603
2003 batch 67 603 612
2003 batch 68 612 621
2003 batch 69 621 630
2003 batch 70 630 639
2003 batch 71 639 648
2003 batch 72 648 657
2003 batch 73 657 666
2003 batch 74 666 675
2003 batch 75 675 684
2003 batch 76 684 693
2003 batch 77 693 702
2003 batch 78 702 711
2003 batch 79 711 720
2003 batch 80 720 729
2003 batch 81 729 738
2003 batch 82 738 747
2003 batch 83 747 756
2003 batch 84 756 765
2003 batch 85 765 774
2003 batch 86 774 783
2003 batch 87 783 792
2003 batch 88 792 801
2003 batch 89 801 810
2003 batch 90 810 819
2003 batch 91 819 828
2003 batch 92 828 837
2003 batch 93 837 846
2003 batch 94 846 855
2003 batch 95 855 864
2003 batch 96 864 873
2003 batch 97 873 882
2003 batch 98 882 891
2003 batch 99 891 900
2003 batch 100 900 909
2003 batch 101 909 918
2003 batch 102 918 927
2003 batch 103 927 936
2003 batch 104 936 945
2003 batch 105 945 954
2003 batch 106 954 963
2003 batch 107 963 972
2003 batch 108 972 981
2003 batch 109 981 990


2006 batch 108 972 981
2006 batch 109 981 990
2006 batch 110 990 999
2007 batch 0 0 9
2007 batch 1 9 18
2007 batch 2 18 27
2007 batch 3 27 36
2007 batch 4 36 45
2007 batch 5 45 54
2007 batch 6 54 63
2007 batch 7 63 72
2007 batch 8 72 81
2007 batch 9 81 90
2007 batch 10 90 99
2007 batch 11 99 108
2007 batch 12 108 117
2007 batch 13 117 126
2007 batch 14 126 135
2007 batch 15 135 144
2007 batch 16 144 153
2007 batch 17 153 162
2007 batch 18 162 171
2007 batch 19 171 180
2007 batch 20 180 189
2007 batch 21 189 198
2007 batch 22 198 207
2007 batch 23 207 216
2007 batch 24 216 225
2007 batch 25 225 234
2007 batch 26 234 243
2007 batch 27 243 252
2007 batch 28 252 261
2007 batch 29 261 270
2007 batch 30 270 279
2007 batch 31 279 288
2007 batch 32 288 297
2007 batch 33 297 306
2007 batch 34 306 315
2007 batch 35 315 324
2007 batch 36 324 333
2007 batch 37 333 342
2007 batch 38 342 351
2007 batch 39 351 360
2007 batch 40 360 369
2007 batch 41 369 378
2007 batch 42 378 387
2007 batch 43 387 396

2010 batch 42 378 387
2010 batch 43 387 396
2010 batch 44 396 405
2010 batch 45 405 414
2010 batch 46 414 423
2010 batch 47 423 432
2010 batch 48 432 441
2010 batch 49 441 450
2010 batch 50 450 459
2010 batch 51 459 468
2010 batch 52 468 477
2010 batch 53 477 486
2010 batch 54 486 495
2010 batch 55 495 504
2010 batch 56 504 513
2010 batch 57 513 522
2010 batch 58 522 531
2010 batch 59 531 540
2010 batch 60 540 549
2010 batch 61 549 558
2010 batch 62 558 567
2010 batch 63 567 576
2010 batch 64 576 585
2010 batch 65 585 594
2010 batch 66 594 603
2010 batch 67 603 612
2010 batch 68 612 621
2010 batch 69 621 630
2010 batch 70 630 639
2010 batch 71 639 648
2010 batch 72 648 657
2010 batch 73 657 666
2010 batch 74 666 675
2010 batch 75 675 684
2010 batch 76 684 693
2010 batch 77 693 702
2010 batch 78 702 711
2010 batch 79 711 720
2010 batch 80 720 729
2010 batch 81 729 738
2010 batch 82 738 747
2010 batch 83 747 756
2010 batch 84 756 765
2010 batch 85 765 774
2010 batch 86 774 783
2010 batch

2013 batch 85 765 774
2013 batch 86 774 783
2013 batch 87 783 792
2013 batch 88 792 801
2013 batch 89 801 810
2013 batch 90 810 819
2013 batch 91 819 828
2013 batch 92 828 837
2013 batch 93 837 846
2013 batch 94 846 855
2013 batch 95 855 864
2013 batch 96 864 873
2013 batch 97 873 882
2013 batch 98 882 891
2013 batch 99 891 900
2013 batch 100 900 909
2013 batch 101 909 918
2013 batch 102 918 927
2013 batch 103 927 936
2013 batch 104 936 945
2013 batch 105 945 954
2013 batch 106 954 963
2013 batch 107 963 972
2013 batch 108 972 981
2013 batch 109 981 990
2013 batch 110 990 999
2014 batch 0 0 9
2014 batch 1 9 18
2014 batch 2 18 27
2014 batch 3 27 36
2014 batch 4 36 45
2014 batch 5 45 54
2014 batch 6 54 63
2014 batch 7 63 72
2014 batch 8 72 81
2014 batch 9 81 90
2014 batch 10 90 99
2014 batch 11 99 108
2014 batch 12 108 117
2014 batch 13 117 126
2014 batch 14 126 135
2014 batch 15 135 144
2014 batch 16 144 153
2014 batch 17 153 162
2014 batch 18 162 171
2014 batch 19 171 180
2014 batch 20

# Exract MODIS LAI fPAR

In [38]:
#product = 'MODIS/006/MCD15A3H'  
product = 'MODIS/061/MCD15A3H'
bands = ['Lai', 'Fpar', 'FparLai_QC']

In [41]:
b =9
for yy in range(2002, period_y[1] + 1):
    start = '%s-01-01' % yy
    end = '%s-01-01' % (yy + 1)
    c = []
    for ix in range(np.int(len(locations)/b +1)):
        print(yy, 'batch', ix)
        
        pts = ee.FeatureCollection([ee.Feature(ee.Geometry.Point([lon, lat])) \
                    for ix, (lon, lat) in enumerate(locations[ix*b: ix*b+b])])
        pts = pts.map(get_buffer)
        ee_data = extract_data_modis_buffer()
        
        keys = ['Lat', 'Lon', 'timestamp', 'year', 'month', 'doy']
        for band in bands:
            keys.append(band)
        
        ee_data = convert_to_df(ee_data, keys)
        c.append(ee_data)
        
    df = pd.concat(c)
    df = df.reset_index()
    df = df.drop(['index', ], axis=1)
    df.to_csv(os.path.join(data_dir, 'grid_data_extract/MCD15A3H061_extract_buffer', 'modis_sites_%s.csv' % (yy)))

2002 batch 0
2002 batch 1
2002 batch 2
2002 batch 3
2002 batch 4
2002 batch 5
2002 batch 6
2002 batch 7
2002 batch 8
2002 batch 9
2002 batch 10
2002 batch 11
2002 batch 12
2002 batch 13
2002 batch 14
2002 batch 15
2002 batch 16
2002 batch 17
2002 batch 18
2002 batch 19
2002 batch 20
2002 batch 21
2002 batch 22
2002 batch 23
2002 batch 24
2002 batch 25
2002 batch 26
2002 batch 27
2002 batch 28
2002 batch 29
2002 batch 30
2002 batch 31
2002 batch 32
2002 batch 33
2002 batch 34
2002 batch 35
2002 batch 36
2002 batch 37
2002 batch 38
2002 batch 39
2002 batch 40
2002 batch 41
2002 batch 42
2002 batch 43
2002 batch 44
2002 batch 45
2002 batch 46
2002 batch 47
2002 batch 48
2002 batch 49
2002 batch 50
2002 batch 51
2002 batch 52
2002 batch 53
2002 batch 54
2002 batch 55
2002 batch 56
2002 batch 57
2002 batch 58
2002 batch 59
2002 batch 60
2002 batch 61
2002 batch 62
2002 batch 63
2002 batch 64
2002 batch 65
2002 batch 66
2002 batch 67
2002 batch 68
2002 batch 69
2002 batch 70
2002 batch 71
20

2007 batch 31
2007 batch 32
2007 batch 33
2007 batch 34
2007 batch 35
2007 batch 36
2007 batch 37
2007 batch 38
2007 batch 39
2007 batch 40
2007 batch 41
2007 batch 42
2007 batch 43
2007 batch 44
2007 batch 45
2007 batch 46
2007 batch 47
2007 batch 48
2007 batch 49
2007 batch 50
2007 batch 51
2007 batch 52
2007 batch 53
2007 batch 54
2007 batch 55
2007 batch 56
2007 batch 57
2007 batch 58
2007 batch 59
2007 batch 60
2007 batch 61
2007 batch 62
2007 batch 63
2007 batch 64
2007 batch 65
2007 batch 66
2007 batch 67
2007 batch 68
2007 batch 69
2007 batch 70
2007 batch 71
2007 batch 72
2007 batch 73
2007 batch 74
2007 batch 75
2007 batch 76
2007 batch 77
2007 batch 78
2007 batch 79
2007 batch 80
2007 batch 81
2007 batch 82
2007 batch 83
2007 batch 84
2007 batch 85
2007 batch 86
2007 batch 87
2007 batch 88
2007 batch 89
2007 batch 90
2007 batch 91
2007 batch 92
2007 batch 93
2007 batch 94
2007 batch 95
2007 batch 96
2007 batch 97
2007 batch 98
2007 batch 99
2007 batch 100
2007 batch 101
2007

2012 batch 61
2012 batch 62
2012 batch 63
2012 batch 64
2012 batch 65
2012 batch 66
2012 batch 67
2012 batch 68
2012 batch 69
2012 batch 70
2012 batch 71
2012 batch 72
2012 batch 73
2012 batch 74
2012 batch 75
2012 batch 76
2012 batch 77
2012 batch 78
2012 batch 79
2012 batch 80
2012 batch 81
2012 batch 82
2012 batch 83
2012 batch 84
2012 batch 85
2012 batch 86
2012 batch 87
2012 batch 88
2012 batch 89
2012 batch 90
2012 batch 91
2012 batch 92
2012 batch 93
2012 batch 94
2012 batch 95
2012 batch 96
2012 batch 97
2012 batch 98
2012 batch 99
2012 batch 100
2012 batch 101
2012 batch 102
2012 batch 103
2012 batch 104
2012 batch 105
2012 batch 106
2012 batch 107
2012 batch 108
2012 batch 109
2012 batch 110
2013 batch 0
2013 batch 1
2013 batch 2
2013 batch 3
2013 batch 4
2013 batch 5
2013 batch 6
2013 batch 7
2013 batch 8
2013 batch 9
2013 batch 10
2013 batch 11
2013 batch 12
2013 batch 13
2013 batch 14
2013 batch 15
2013 batch 16
2013 batch 17
2013 batch 18
2013 batch 19
2013 batch 20
2013 

# Exract MODIS continuous vegetation cover

In [32]:
product = 'MODIS/006/MOD44B'  
bands = ['Percent_Tree_Cover', 'Percent_NonTree_Vegetation', 'Percent_NonVegetated', 'Quality']

In [33]:
c = []
pts = ee.FeatureCollection([ee.Feature(ee.Geometry.Point([lon, lat])) \
                for ix, (lon, lat) in enumerate(locations)])  
for yy in range(2002, 2021):
    start = '%s-01-01' % yy
    end = '%s-01-01' % (yy + 1)
    print(yy, )

    ee_data = extract_data()
    #keys = list(ee_data.getInfo()['features'][0]['properties'].keys())
    keys = ['Lat', 'Lon', 'timestamp', 'year', 'month', 'doy']
    for band in bands:
        keys.append(band)
    ee_data = convert_to_df(ee_data, keys)
    c.append(ee_data)
df = pd.concat(c)
df = df.reset_index()
df = df.drop(['index', ], axis=1)
df.to_csv(os.path.join(data_dir, 'grid_data_extract/MOD44B_extract', 'modis_vc_sites.csv'))

2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


# Extract OpenLandMap

In [19]:
product = "OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02"  
bands = ['b0', 'b10', 'b30', 'b60', 'b100', 'b200']
output = extract_pt_by_pt(locations, product, bands)
print(output)

           Lon        Lat   b0  b10  b30  b60  b100  b200
0    32.931111  34.929444  9.0  9.0  9.0  9.0   9.0   9.0
1    32.931111  34.929444  9.0  9.0  9.0  9.0   9.0   9.0
2    32.830556  34.952222  9.0  9.0  9.0  6.0   6.0   6.0
3    33.050000  35.013889  9.0  9.0  6.0  6.0   6.0   6.0
4    32.627500  35.020000  9.0  9.0  6.0  6.0   6.0   6.0
..         ...        ...  ...  ...  ...  ...   ...   ...
991  24.236111  67.999444  9.0  9.0  9.0  9.0   9.0   9.0
992  19.427500  68.995278  9.0  9.0  9.0  9.0   9.0   9.0
993  30.036111  69.454167  9.0  9.0  9.0  9.0   9.0   9.0
994  28.898056  69.583056  NaN  NaN  NaN  NaN   NaN   NaN
995  26.958056  69.738889  9.0  9.0  9.0  9.0   9.0   9.0

[996 rows x 8 columns]


In [20]:
output.to_csv(os.path.join(data_dir, 'grid_data_extract', 'soiltex_extract.csv'))

In [21]:
product = "OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01"
bands = ['b0', 'b10', 'b30', 'b60', 'b100', 'b200']
output = extract_pt_by_pt(locations, product, bands)
print(output)

           Lon        Lat    b0   b10   b30   b60  b100  b200
0    32.931111  34.929444  19.0  17.0  17.0  16.0  15.0  15.0
1    32.931111  34.929444  19.0  17.0  17.0  16.0  15.0  15.0
2    32.830556  34.952222  22.0  19.0  19.0  18.0  18.0  17.0
3    33.050000  35.013889  20.0  19.0  17.0  19.0  19.0  17.0
4    32.627500  35.020000  25.0  24.0  24.0  22.0  22.0  21.0
..         ...        ...   ...   ...   ...   ...   ...   ...
991  24.236111  67.999444  34.0  31.0  27.0  25.0  24.0  25.0
992  19.427500  68.995278  44.0  41.0  38.0  34.0  30.0  30.0
993  30.036111  69.454167  37.0  34.0  32.0  30.0  29.0  29.0
994  28.898056  69.583056   NaN   NaN   NaN   NaN   NaN   NaN
995  26.958056  69.738889  36.0  35.0  34.0  33.0  31.0  31.0

[996 rows x 8 columns]


In [22]:
output.to_csv(os.path.join(data_dir, 'grid_data_extract', 'soilwatercontent_extract.csv'))

In [23]:
product = 'OpenLandMap/PNV/PNV_BIOME-TYPE_BIOME00K_C/v01'
bands = ['biome_type', ]
output = extract_pt_by_pt(locations, product, bands)
print(output)

           Lon        Lat  biome_type
0    32.931111  34.929444         4.0
1    32.931111  34.929444         4.0
2    32.830556  34.952222         4.0
3    33.050000  35.013889         4.0
4    32.627500  35.020000         4.0
..         ...        ...         ...
991  24.236111  67.999444        15.0
992  19.427500  68.995278        15.0
993  30.036111  69.454167        15.0
994  28.898056  69.583056        15.0
995  26.958056  69.738889        15.0

[996 rows x 3 columns]


In [24]:
output.to_csv(os.path.join(data_dir, 'grid_data_extract', 'biome_extract.csv'))

# Extract elevation

In [25]:
product = "USGS/GTOPO30"  
bands = ['elevation', ]
output = extract_pt_by_pt(locations, product, bands)
print(output)

           Lon        Lat  elevation
0    32.931111  34.929444     1340.0
1    32.931111  34.929444     1340.0
2    32.830556  34.952222     1425.0
3    33.050000  35.013889      743.0
4    32.627500  35.020000      757.0
..         ...        ...        ...
991  24.236111  67.999444      345.0
992  19.427500  68.995278      200.0
993  30.036111  69.454167       33.0
994  28.898056  69.583056      105.0
995  26.958056  69.738889      140.0

[996 rows x 3 columns]


In [26]:
output.to_csv(os.path.join(data_dir, 'grid_data_extract', 'GTOPO30_extract.csv'))

In [27]:
#https://developers.google.com/earth-engine/datasets/catalog/USGS_GTOPO30

# Extract forest canopy height

In [58]:
product = 'NASA/JPL/global_forest_canopy_height_2005'  
bands = ['1', ]
output = extract_pt_by_pt(locations, product, bands)
print(output)

           Lon        Lat   1
0    32.931111  34.929444  20
1    32.931111  34.929444  20
2    32.830556  34.952222   0
3    33.050000  35.013889   0
4    32.627500  35.020000  20
..         ...        ...  ..
991  24.236111  67.999444  15
992  19.427500  68.995278  16
993  30.036111  69.454167  13
994  28.898056  69.583056  14
995  26.958056  69.738889   0

[996 rows x 3 columns]


In [59]:
output.to_csv(os.path.join(data_dir, 'grid_data_extract', 'global_forest_canopy_height_2005_extract.csv'))